In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './prepped_data/'

In [3]:
%%time
main_data_df = pd.read_csv(basedir + 'main_data_baseline_checkpoint_2.csv')
main_data_df.shape

Wall time: 5.9 s


(4441256, 17)

In [4]:
%%time
land_plans_df = pd.read_csv(basedir + 'land_plans_ver_2.csv')
land_plans_df.shape

<string>:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,27

Wall time: 14.4 s


(874028, 272)

In [5]:
%%time
bdinfo_df = pd.read_csv(basedir + 'bdinfo_floor_info_features_full_2.csv')
bdinfo_df.shape

Wall time: 13.6 s


(1406750, 252)

In [6]:
%%time
land_prices_df = pd.read_csv(basedir + 'land_prices_ver_2.csv')
land_prices_df.shape

Wall time: 562 ms


(1594565, 4)

In [7]:
%%time
land_specs_df = pd.read_csv(basedir + 'land_specs_ver_2.csv')
land_specs_df.shape

Wall time: 831 ms


(867710, 11)

In [8]:
%%time
floor_df = pd.read_csv(basedir + 'living_area_per_room_full.csv')
floor_df.shape

Wall time: 704 ms


(1475121, 6)

In [9]:
land_plans_df.fillna('해당사항없음', inplace=True)

In [10]:
dfs_list = [main_data_df, land_plans_df, bdinfo_df, land_prices_df, floor_df, land_specs_df]
for df in dfs_list:
    print(df.shape)

(4441256, 17)
(874028, 272)
(1406750, 252)
(1594565, 4)
(1475121, 6)
(867710, 11)


In [11]:
main_data_df.rename(columns={'전용면적(㎡)':'전용면적'}, inplace=True)

In [12]:
%%time
main_data_df['전용면적_mean'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('mean')['전용면적']
main_data_df['전용면적_min'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('min')['전용면적']
main_data_df['전용면적_max'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('max')['전용면적']

Wall time: 1.53 s


In [13]:
main_data_df = main_data_df[(main_data_df['전월세매매구분'] == '매매')|(main_data_df['전월세매매구분'] == '전세')].drop(columns=['월세(만원)', '도로명주소'])
print(main_data_df.shape)

(3607377, 18)


In [14]:
main_data_df['건물나이'] = main_data_df['년'] - main_data_df['건축년도']
main_data_df['건물나이'].describe()

count    3.607377e+06
mean     1.393074e+01
std      9.846349e+00
min     -1.200000e+01
25%      6.000000e+00
50%      1.300000e+01
75%      2.100000e+01
max      8.900000e+01
Name: 건물나이, dtype: float64

In [15]:
main_data_df['건물나이^2'] = main_data_df['건물나이']**2

In [16]:
main_data_df.drop(columns=['건축년도'], inplace=True)

In [17]:
main_data_df['year_linear'] = (main_data_df['년'] - main_data_df['년'].min()) + ((main_data_df['월']-1)/12)

In [18]:
def month_circular_sine_func(x):
    return math.sin(2*math.pi*((x-1)/12))

def month_circular_cosine_func(x):
    return math.cos(2*math.pi*((x-1)/12))

In [19]:
main_data_df['month_sin'] = main_data_df['월'].apply(month_circular_sine_func)
main_data_df['month_cos'] = main_data_df['월'].apply(month_circular_cosine_func)

In [20]:
main_data_df.drop(columns=['월', '계약일'], inplace=True)

In [21]:
xmax = main_data_df['X좌표'].max()
xmin = main_data_df['X좌표'].min()
ymax = main_data_df['Y좌표'].max()
ymin = main_data_df['Y좌표'].min()

In [22]:
coor_minmax_df = pd.DataFrame({'xmax':xmax, 'xmin':xmin, 'ymax':ymax, 'ymin':ymin}, index=[0])
coor_minmax_df.to_csv('./Training/training_data_ver_10/coor_minmax.csv', index=False)
coor_minmax_df

,xmax,xmin,ymax,ymin
0,971728.508488,938089.149737,1.965830e+06,1.937374e+06


In [23]:
main_data_df['xnorm'] = (main_data_df['X좌표'] - xmin - ((xmax - xmin)/2)) / 10000
main_data_df['ynorm'] = (main_data_df['Y좌표'] - ymin - ((ymax - ymin)/2)) / 10000
main_data_df['x_2nd'] = main_data_df['xnorm']**2 / 2
main_data_df['y_2nd'] = main_data_df['ynorm']**2 / 2
main_data_df['x_sin'] = main_data_df['xnorm'].apply(np.sin)
main_data_df['y_sin'] = main_data_df['ynorm'].apply(np.sin)
main_data_df['x_cos'] = main_data_df['xnorm'].apply(np.cos)
main_data_df['y_cos'] = main_data_df['ynorm'].apply(np.cos)
main_data_df['xy'] = main_data_df['xnorm'] * main_data_df['ynorm']

In [24]:
main_data_df.drop(columns=['X좌표', 'Y좌표'], inplace=True)

In [25]:
main_data_df['trade_type'] = main_data_df['건물종류'] + '_' + main_data_df['전월세매매구분']
main_data_df.drop(columns=['건물종류', '전월세매매구분'], inplace=True)

In [26]:
main_data_df.head()

,전용면적,금액(만원),층,가격/면적,년,target_log_transformed,target/area_log_transformed,지번주소,전용면적_mean,전용면적_min,전용면적_max,건물나이,건물나이^2,year_linear,month_sin,month_cos,xnorm,ynorm,x_2nd,y_2nd,x_sin,y_sin,x_cos,y_cos,xy,trade_type
0,77.75,57000,2,733.118971,2013,10.950807,6.597308,서울특별시 강남구 개포동 655-2,77.75,77.75,77.75,25,625,7.666667,-0.866025,-0.500000,0.556491,-0.904896,0.154841,0.409418,0.52821,-0.786361,0.849114,0.617768,-0.503567,아파트_매매
1,77.75,57000,2,733.118971,2013,10.950807,6.597308,서울특별시 강남구 개포동 655-2,77.75,77.75,77.75,25,625,7.916667,-0.500000,0.866025,0.556491,-0.904896,0.154841,0.409418,0.52821,-0.786361,0.849114,0.617768,-0.503567,아파트_매매
2,77.75,55000,7,707.395498,2014,10.915088,6.561590,서울특별시 강남구 개포동 655-2,77.75,77.75,77.75,26,676,8.000000,0.000000,1.000000,0.556491,-0.904896,0.154841,0.409418,0.52821,-0.786361,0.849114,0.617768,-0.503567,아파트_매매
3,77.75,59500,4,765.273312,2014,10.993732,6.640233,서울특별시 강남구 개포동 655-2,77.75,77.75,77.75,26,676,8.833333,-0.866025,0.500000,0.556491,-0.904896,0.154841,0.409418,0.52821,-0.786361,0.849114,0.617768,-0.503567,아파트_매매
4,77.75,75000,5,964.630225,2016,11.225243,6.871745,서울특별시 강남구 개포동 655-2,77.75,77.75,77.75,28,784,10.166667,0.866025,0.500000,0.556491,-0.904896,0.154841,0.409418,0.52821,-0.786361,0.849114,0.617768,-0.503567,아파트_매매


In [27]:
floor_df.head()

,지번주소,주소별_기준년월_총세대수합_최대,전체연면적,세대별연면적,기준년도,사용승인년도
0,서울특별시 강남구 개포동 1163-4,20,540.44,27.022,2014.0,2013.0
1,서울특별시 강남구 개포동 1163-4,20,540.44,27.022,2015.0,2013.0
2,서울특별시 강남구 개포동 1163-4,20,540.44,27.022,2016.0,2013.0
3,서울특별시 강남구 개포동 1163-4,20,540.44,27.022,2017.0,2013.0
4,서울특별시 강남구 개포동 1163-4,20,540.44,27.022,2018.0,2013.0


In [28]:
floor_df['년'] = floor_df['기준년도'].astype('Int64')
floor_df.head()

,지번주소,주소별_기준년월_총세대수합_최대,전체연면적,세대별연면적,기준년도,사용승인년도,년
0,서울특별시 강남구 개포동 1163-4,20,540.44,27.022,2014.0,2013.0,2014
1,서울특별시 강남구 개포동 1163-4,20,540.44,27.022,2015.0,2013.0,2015
2,서울특별시 강남구 개포동 1163-4,20,540.44,27.022,2016.0,2013.0,2016
3,서울특별시 강남구 개포동 1163-4,20,540.44,27.022,2017.0,2013.0,2017
4,서울특별시 강남구 개포동 1163-4,20,540.44,27.022,2018.0,2013.0,2018


In [29]:
floor_df.drop(columns=['기준년도', '사용승인년도'], inplace=True)

In [30]:
floor_df.rename(columns={'주소별_기준년월_총세대수합_최대':'총호수'}, inplace=True)

In [31]:
floor_features_df = floor_df[['지번주소', '년', '세대별연면적', '총호수']].drop_duplicates(subset=['지번주소', '년'])

In [32]:
merge_df = main_data_df.merge(floor_features_df, on=['지번주소', '년'])
merge_df.shape

(2204817, 28)

In [33]:
bdinfo_df.rename(columns={'기준년도':'년'}, inplace=True)
bdinfo_df.drop(columns=['사용승인년도'], inplace=True)

In [34]:
merge2_df = merge_df.merge(bdinfo_df, on=['지번주소', '년'])
merge2_df.shape

(2204628, 277)

In [35]:
merge3_df = merge2_df.merge(land_specs_df, on=['지번주소', '년'])
merge3_df.shape

(2147673, 286)

In [36]:
merge3_df = merge3_df.merge(land_plans_df, on=['지번주소', '년'])
merge3_df.shape

(1649776, 556)

In [37]:
merge3_df['공시지가'] = merge3_df['공시지가'] / 10000
merge3_df.rename(columns={'공시지가':'공시지가(만원)'}, inplace=True)

In [38]:
merge3_df = merge3_df.merge(land_prices_df[['지번주소', '년', '표준지여부']], on=['지번주소', '년'])
merge3_df.shape

(1649775, 557)

In [39]:
merge3_df.drop(columns=['지번주소'], inplace=True)

In [40]:
merge3_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','') for col in merge3_df.columns]

In [41]:
land_plans_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','') for col in land_plans_df.columns]

In [42]:
lp_catcols = land_plans_df.columns.tolist()[2:]

In [43]:
ls_catcols = [col for col in land_specs_df.columns if col not in ['지번주소', '년', '토지면적', '공시지가']]
ls_catcols

['지목명', '용도지역명1', '용도지역명2', '토지이동상황', '지형높이', '지형형상', '도로접면']

In [44]:
merge3_df.head()

,전용면적,금액(만원),층,가격면적,년,target_log_transformed,targetarea_log_transformed,전용면적_mean,전용면적_min,전용면적_max,건물나이,건물나이^2,year_linear,month_sin,month_cos,xnorm,ynorm,x_2nd,y_2nd,x_sin,y_sin,x_cos,y_cos,xy,trade_type,세대별연면적,총호수,주_용도_부대시설_있음,주_용도_주차장_있음,주_용도_복리시설_있음,주_용도_사무소_있음,주_용도_소매점_있음,주_용도_차고_있음,주_용도_일반음식점_있음,주_용도_기타제1종근생활시설_있음,주_용도_기타제2종근생활시설_있음,주_용도_기타근린생활시설_있음,주_용도_기타창고시설_있음,주_용도_기타판매시설_있음,주_용도_기타일반업무시설_있음,주_용도_기타사무소_있음,주_용도_유치원_있음,주_용도_노인복지시설_있음,주_용도_치과의원_있음,주_용도_의원_있음,주_용도_제조업소_있음,주_용도_대피소_있음,주_용도_휴게음식점_있음,주_용도_학원_있음,주_용도_생활편익시설_있음,주_용도_세탁소_있음,주_용도_창고_있음,주_용도_독서실_있음,주_용도_기타노유자시설_있음,주_용도_부동산중개업소_있음,주_용도_기원_있음,주_용도_기타교육연구시설_있음,주_용도_이(미)용원_있음,주_용도_멀티미디어문화컨텐츠설비제공업소_있음,주_용도_기타자동차관련시설_있음,주_용도_어린이집_있음,주_용도_한의원_있음,주_용도_금융업소_있음,주_용도_일반게임제공업의시설_있음,주_용도_파출소_있음,주_용도_인터넷컴퓨터게임시설제공업소_있음,주_용도_상점_있음,주_용도_게임제공업소_있음,주_용도_기타교육연구및복지시설_있음,주_용도_출판사_있음,주_용도_일반목욕장_있음,주_용도_공중화장실_있음,주_용도_부동산중개사무소_있음,주_용도_미용원_있음,주_용도_기타아동관련시설_있음,주_용도_기타공장_있음,주_용도_수리점_있음,주_용도_영유아보육시설_있음,주_용도_연구소_있음,주_용도_체육도장_있음,주_용도_지역아동센터_있음,주_용도_체육장_있음,주_용도_기타종교집회장_있음,주_용도_기타 운동시설_있음,주_용도_노래연습장_있음,주_용도_당구장_있음,주_용도_기타의료시설_있음,주_용도_아동복지시설_있음,주_용도_의약품도매점_있음,주_용도_교회_있음,주_용도_단란주점_있음,주_용도_조산원_있음,주_용도_침술원_있음,주_용도_표구점_있음,주_용도_목욕장_있음,주_용도_기타운동시설_있음,주_용도_기타문화및집회시설_있음,주_용도_기타전시장_있음,주_용도_사진관_있음,주_용도_기타일반숙박시설_있음,주_용도_체력단련장_있음,주_용도_가족호텔_있음,주_용도_조산소_있음,주_용도_접골원_있음,주_용도_기타생활권수련시설_있음,주_용도_기타공연장_있음,주_용도_탁구장_있음,주_용도_시장_있음,주_용도_직업훈련소_있음,주_용도_기타집회장_있음,주_용도_골프연습장_있음,주_용도_교육(연수)원_있음,주_용도_결혼상담소_있음,주_용도_인터넷컴퓨터게임시설제공업의시설_있음,주_용도_기타종교시설_있음,주_용도_사회복지시설_있음,주_용도_기타공공시설_있음,주_용도_기타판매및영업시설_있음,주_용도_사찰_있음,주_용도_의약품판매소_있음,주_용도_의료기기판매소_있음,주_용도_청소년게임제공업의시설_있음,주_용도_이용원_있음,주_용도_서점(1종근.생미해당)_있음,주_용도_기타위락시설_있음,주_용도_마을공동시설_있음,주_용도_기타위험물저장처리시설_있음,주_용도_변전소_있음,주_용도_병원_있음,주_용도_동물병원_있음,주_용도_유흥주점_있음,주_용도_예식장_있음,주_용도_일반업무시설_있음,주_용도_기타발전시설_있음,주_용도_수퍼마켓_있음,주_용도_공공도서관_있음,주_용도_자동차영업소_있음,주_용도_제1종근린생활시설_있음,주_용도_분뇨처리시설_있음,주_용도_기타소매시장_있음,주_용도_온실_있음,주_용도_소매시장_있음,주_용도_안마시술소_있음,주_용도_야외극장_있음,주_용도_창고시설_있음,주_용도_에어로빅장_있음,주_용도_제2종근린생활시설_있음,주_용도_극장_있음,주_용도_한방병원_있음,주_용도_극장(영화관)_있음,주_용도_기타병원_있음,주_용도_제과점_있음,주_용도_교육연구시설_있음,주_용도_대형백화점_있음,주_용도_기타분뇨쓰레기처리시설_있음,주_용도_총포판매사_있음,주_용도_일반공장_있음,주_용도_기타공공업무시설_있음,주_용도_정수장_있음,주_용도_일반창고_있음,주_용도_운동장시설_있음,주_용도_여관_있음,주_용도_대형점_있음,주_용도_고시원_있음,주_용도_유원시설업의시설_있음,주_용도_무도장_있음,주_용도_치과병원_있음,주_용도_안마원_있음,주_용도_관광호텔_있음,주_용도_도서관_있음,주_용도_비디오물감상실_있음,주_용도_볼링장_있음,주_용도_교육원_있음,주_용도_마을공동작업소_있음,주_용도_산부인과병원_있음,주_용도_대학_있음,주_용도_주점영업_있음,주_용도_소개업_있음,주_용도_요양병원_있음,주_용도_수족관_있음,주_용도_매매장_있음,주_용도_테니스장_있음,주_용도_동사무소_있음,주_용도_무도장(학원)_있음,주_용도_대학교_있음,주_용도_호텔_있음,주_용도_운동시설_있음,주_용도_복합유통게임제공업의시설_있음,주_용도_영화관_있음,주_용도_기숙사_있음,주_용도_체육관_있음,주_용도_도매시장_있음,주_용도_정비공장_있음,주_용도_복합유통게임제공업소_있음,주_용도_회의장_있음,주_용도_총포판매소_있음,주_용도_미술관_있음,주_용도_백화점_있음,주_용도_기타학교_있음,주_용도_공관_있음,주_용도_군사시설_있음,주_용도_방송국_있음,주_용도_기타제2종근린생활시설_있음,주_용도_기타제1종근린생활시설_있음,주_용도_기타동식물관련시설_있음,주_용도_고압가스충전저장소_있음,주_용도_검사장_있음,주_용도_초등학교_있음,주_용도_기타관광숙박시설_있음,주_용도_세차장_있음,주_용도_성당_있음,주_용도_축사_있음,주_용도_기타관광휴게시설_있음,주_용도_위험물취급소_있음,주_용도_석유판매소_있음,주_용도_물놀이형시설_있음,주_용도_보건소_있음,주_용도_고등학교_있음,주_용도_유해공장_있음,주_용도_주유소_있음,주_용도_다중생활시설_있음,주_용도_일반숙박시설_있음,주_용도_호스텔_있음,주_용도_박물관_있음,주_용도_생활숙박시설_있음,주_용도_청소년문화의집_있음,주_용도_근린생활시설_있음,주_용도_산후조리원_있음,주_용도_교습소_있음,주_용도_청소년게임제공업소_있음,주_용도_액화가스판매소_있음,주_용도_야외음악당_있음,주_용도_신문사_있음,주_용도_음악당_있음,주_용도_수도원_있음,주_용도_냉동창고_있음,주_용도_놀이형시설_있음,주_용도_기타게임시설_있음,주_용도_제실_있음,주_용도_자치단체청사_있음,주_용도_비디오물소극장_있음,주_용도_국방ㆍ군사시설_있음,주_용도_지구대_있음,주_용도_기도원_있음,주_용도_롤러스케이트장_있음,주_용도_소형호텔_있음,주_용도_수영장_있음,주_용도_동물미용실_있음,주_용도_도료류판매소_있음,주_용도_물품 제조공장_있음,...,공공문화체육시설미분류,공공시설구역,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공원,공원마을지구(공원집단시설지구),공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항소음피해예상지역,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광장,교육환경보호구역,교통광장,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,근린공원,근린광장,근린상업지역,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타문화시설,기타보건위생시설,기타수도시설,기타시장시설,기타용도지역지구기타,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타주차장시설,기타철도시설,기타폐기물처리시설,노외주차장,녹지,대공방어협조구역,대공방어협조구역(위탁고도54-236m),대공방어협조구역(위탁고도77-257m),대로1류(폭 35M~40M),대로2류(폭 30M~35M),대로3류(폭 25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획 입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도

In [45]:
merge3_df.drop(columns=['년'], inplace=True)

In [46]:
cat_cols = ['trade_type', '표준지여부'] + lp_catcols + ls_catcols + ['최빈용도']

In [47]:
for col in tqdm(cat_cols):
    col_map = {}
    col_map_df = pd.DataFrame({'cat':merge3_df[col].unique(),'map':[i for i in range(len(merge3_df[col].unique()))]})
    for i in range(col_map_df.shape[0]):
        col_map[col_map_df['cat'][i]] = col_map_df['map'][i]
    merge3_df[col] = merge3_df[col].map(col_map)
    col_map_df.to_csv('./Training/training_data_ver_10/feature_maps/{}_map.csv'.format(col))

100%|████████████████████████████████████████████████████████████████████████████████| 280/280 [03:47<00:00,  1.23it/s]


In [48]:
cat_cols_df = pd.DataFrame({'colname':cat_cols})
cat_cols_df.head()

,colname
0,trade_type
1,표준지여부
2,(한강)오염행위 제한지역
3,(한강)폐기물매립시설 설치제한지역
4,가로구역별 최고높이 제한지역


In [49]:
cat_cols_df.to_csv('./Training/training_data_ver_10/cat_cols.csv', index=False)

In [50]:
cols_to_drop = []
for col in tqdm(merge3_df.columns):
    try:
        if merge3_df[col].std() == 0:
            cols_to_drop.append(col)
        else:
            pass
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████| 555/555 [00:07<00:00, 70.79it/s]


In [51]:
len(cols_to_drop)

26

In [52]:
cols_to_drop

['주_용도_기타일반숙박시설_있음',
 '주_용도_일반업무시설_있음',
 '주_용도_제1종근린생활시설_있음',
 '주_용도_복합유통게임제공업의시설_있음',
 '주_용도_방송국_있음',
 '주_용도_기타동식물관련시설_있음',
 '주_용도_검사장_있음',
 '주_용도_위험물취급소_있음',
 '주_용도_석유판매소_있음',
 '주_용도_물놀이형시설_있음',
 '주_용도_유해공장_있음',
 '주_용도_주유소_있음',
 '주_용도_일반숙박시설_있음',
 '주_용도_박물관_있음',
 '주_용도_근린생활시설_있음',
 '주_용도_액화가스판매소_있음',
 '주_용도_신문사_있음',
 '주_용도_냉동창고_있음',
 '주_용도_소형호텔_있음',
 '주_용도_물품 제조공장_있음',
 '주_용도_물품 인쇄공장_있음',
 '구조_라멘조_fraction',
 '경관광장',
 '기타유통및공급시설',
 '시·도 생태·경관보전지역',
 '폐기물처리및재활용시설']

In [53]:
merge3_df.drop(columns=cols_to_drop, inplace=True)

In [54]:
merge3_df.head()

,전용면적,금액(만원),층,가격면적,target_log_transformed,targetarea_log_transformed,전용면적_mean,전용면적_min,전용면적_max,건물나이,건물나이^2,year_linear,month_sin,month_cos,xnorm,ynorm,x_2nd,y_2nd,x_sin,y_sin,x_cos,y_cos,xy,trade_type,세대별연면적,총호수,주_용도_부대시설_있음,주_용도_주차장_있음,주_용도_복리시설_있음,주_용도_사무소_있음,주_용도_소매점_있음,주_용도_차고_있음,주_용도_일반음식점_있음,주_용도_기타제1종근생활시설_있음,주_용도_기타제2종근생활시설_있음,주_용도_기타근린생활시설_있음,주_용도_기타창고시설_있음,주_용도_기타판매시설_있음,주_용도_기타일반업무시설_있음,주_용도_기타사무소_있음,주_용도_유치원_있음,주_용도_노인복지시설_있음,주_용도_치과의원_있음,주_용도_의원_있음,주_용도_제조업소_있음,주_용도_대피소_있음,주_용도_휴게음식점_있음,주_용도_학원_있음,주_용도_생활편익시설_있음,주_용도_세탁소_있음,주_용도_창고_있음,주_용도_독서실_있음,주_용도_기타노유자시설_있음,주_용도_부동산중개업소_있음,주_용도_기원_있음,주_용도_기타교육연구시설_있음,주_용도_이(미)용원_있음,주_용도_멀티미디어문화컨텐츠설비제공업소_있음,주_용도_기타자동차관련시설_있음,주_용도_어린이집_있음,주_용도_한의원_있음,주_용도_금융업소_있음,주_용도_일반게임제공업의시설_있음,주_용도_파출소_있음,주_용도_인터넷컴퓨터게임시설제공업소_있음,주_용도_상점_있음,주_용도_게임제공업소_있음,주_용도_기타교육연구및복지시설_있음,주_용도_출판사_있음,주_용도_일반목욕장_있음,주_용도_공중화장실_있음,주_용도_부동산중개사무소_있음,주_용도_미용원_있음,주_용도_기타아동관련시설_있음,주_용도_기타공장_있음,주_용도_수리점_있음,주_용도_영유아보육시설_있음,주_용도_연구소_있음,주_용도_체육도장_있음,주_용도_지역아동센터_있음,주_용도_체육장_있음,주_용도_기타종교집회장_있음,주_용도_기타 운동시설_있음,주_용도_노래연습장_있음,주_용도_당구장_있음,주_용도_기타의료시설_있음,주_용도_아동복지시설_있음,주_용도_의약품도매점_있음,주_용도_교회_있음,주_용도_단란주점_있음,주_용도_조산원_있음,주_용도_침술원_있음,주_용도_표구점_있음,주_용도_목욕장_있음,주_용도_기타운동시설_있음,주_용도_기타문화및집회시설_있음,주_용도_기타전시장_있음,주_용도_사진관_있음,주_용도_체력단련장_있음,주_용도_가족호텔_있음,주_용도_조산소_있음,주_용도_접골원_있음,주_용도_기타생활권수련시설_있음,주_용도_기타공연장_있음,주_용도_탁구장_있음,주_용도_시장_있음,주_용도_직업훈련소_있음,주_용도_기타집회장_있음,주_용도_골프연습장_있음,주_용도_교육(연수)원_있음,주_용도_결혼상담소_있음,주_용도_인터넷컴퓨터게임시설제공업의시설_있음,주_용도_기타종교시설_있음,주_용도_사회복지시설_있음,주_용도_기타공공시설_있음,주_용도_기타판매및영업시설_있음,주_용도_사찰_있음,주_용도_의약품판매소_있음,주_용도_의료기기판매소_있음,주_용도_청소년게임제공업의시설_있음,주_용도_이용원_있음,주_용도_서점(1종근.생미해당)_있음,주_용도_기타위락시설_있음,주_용도_마을공동시설_있음,주_용도_기타위험물저장처리시설_있음,주_용도_변전소_있음,주_용도_병원_있음,주_용도_동물병원_있음,주_용도_유흥주점_있음,주_용도_예식장_있음,주_용도_기타발전시설_있음,주_용도_수퍼마켓_있음,주_용도_공공도서관_있음,주_용도_자동차영업소_있음,주_용도_분뇨처리시설_있음,주_용도_기타소매시장_있음,주_용도_온실_있음,주_용도_소매시장_있음,주_용도_안마시술소_있음,주_용도_야외극장_있음,주_용도_창고시설_있음,주_용도_에어로빅장_있음,주_용도_제2종근린생활시설_있음,주_용도_극장_있음,주_용도_한방병원_있음,주_용도_극장(영화관)_있음,주_용도_기타병원_있음,주_용도_제과점_있음,주_용도_교육연구시설_있음,주_용도_대형백화점_있음,주_용도_기타분뇨쓰레기처리시설_있음,주_용도_총포판매사_있음,주_용도_일반공장_있음,주_용도_기타공공업무시설_있음,주_용도_정수장_있음,주_용도_일반창고_있음,주_용도_운동장시설_있음,주_용도_여관_있음,주_용도_대형점_있음,주_용도_고시원_있음,주_용도_유원시설업의시설_있음,주_용도_무도장_있음,주_용도_치과병원_있음,주_용도_안마원_있음,주_용도_관광호텔_있음,주_용도_도서관_있음,주_용도_비디오물감상실_있음,주_용도_볼링장_있음,주_용도_교육원_있음,주_용도_마을공동작업소_있음,주_용도_산부인과병원_있음,주_용도_대학_있음,주_용도_주점영업_있음,주_용도_소개업_있음,주_용도_요양병원_있음,주_용도_수족관_있음,주_용도_매매장_있음,주_용도_테니스장_있음,주_용도_동사무소_있음,주_용도_무도장(학원)_있음,주_용도_대학교_있음,주_용도_호텔_있음,주_용도_운동시설_있음,주_용도_영화관_있음,주_용도_기숙사_있음,주_용도_체육관_있음,주_용도_도매시장_있음,주_용도_정비공장_있음,주_용도_복합유통게임제공업소_있음,주_용도_회의장_있음,주_용도_총포판매소_있음,주_용도_미술관_있음,주_용도_백화점_있음,주_용도_기타학교_있음,주_용도_공관_있음,주_용도_군사시설_있음,주_용도_기타제2종근린생활시설_있음,주_용도_기타제1종근린생활시설_있음,주_용도_고압가스충전저장소_있음,주_용도_초등학교_있음,주_용도_기타관광숙박시설_있음,주_용도_세차장_있음,주_용도_성당_있음,주_용도_축사_있음,주_용도_기타관광휴게시설_있음,주_용도_보건소_있음,주_용도_고등학교_있음,주_용도_다중생활시설_있음,주_용도_호스텔_있음,주_용도_생활숙박시설_있음,주_용도_청소년문화의집_있음,주_용도_산후조리원_있음,주_용도_교습소_있음,주_용도_청소년게임제공업소_있음,주_용도_야외음악당_있음,주_용도_음악당_있음,주_용도_수도원_있음,주_용도_놀이형시설_있음,주_용도_기타게임시설_있음,주_용도_제실_있음,주_용도_자치단체청사_있음,주_용도_비디오물소극장_있음,주_용도_국방ㆍ군사시설_있음,주_용도_지구대_있음,주_용도_기도원_있음,주_용도_롤러스케이트장_있음,주_용도_수영장_있음,주_용도_동물미용실_있음,주_용도_도료류판매소_있음,최빈용도,지붕_(철근)콘크리트_fraction,지붕_기타지붕_fraction,지붕_기와_fraction,지붕_슬레이트_fraction,구조_철근콘크리트구조_fraction,구조_철골철근콘크리트구조_fraction,구조_벽돌구조_fraction,구조_프리케스트콘크리트구조_fraction,구조_경량철골구조_fraction,구조_일반철골구조_fraction,구조_블록구조_fraction,구조_철골콘크리트구조_fraction,구조_일반목구조_fraction,구조_기타조적구조_fraction,구조_기타강구조_fraction,구조_기타구조_fraction,구조_기타철골철근콘크리트구조_fraction,구조_석구조_fraction,구조_강파이프구조_fraction,구조_기타콘크리트구조_fraction,...,고속철도,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공원,공원마을지구(공원집단시설지구),공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항소음피해예상지역,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광장,교육환경보호구역,교통광장,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,근린공원,근린광장,근린상업지역,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타문화시설,기타보건위생시설,기타수도시설,기타시장시설,기타용도지역지구기타,기타용지,기타유통업무설비,기타자동차정류장,기타주차장시설,기타철도시설,기타폐기물처리시설,노외주차장,녹지,대공방어협조구역,대공방어협조구역(위탁고도54-236m),대공방어협조구역(위탁고도77-257m),대로1류(폭 35M~40M),대로2류(폭 30M

In [55]:
%%time
merge3_df.to_csv('./Training/training_data_ver_10/training_data_ver_10.csv', index=False)

Wall time: 7min 53s


In [56]:
land_specs_df.head()

,지번주소,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
0,서울특별시 강남구 개포동 1163-4,2013,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,6290000.0
1,서울특별시 강남구 개포동 1163-4,2014,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,6480000.0
2,서울특별시 강남구 개포동 1163-4,2015,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,6870000.0
3,서울특별시 강남구 개포동 1163-4,2016,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,7125000.0
4,서울특별시 강남구 개포동 1163-4,2017,대,336.1,제3종일반주거지역,지정되지않음,상업기타,평지,세로장방,광대세각,7485000.0
